# Key Features
#### Dynamic NIFTY CSV parsing → avoids ParserError.
#### Dow Jones handled separately via Wikipedia API.
#### Multi-index tagging → symbols in multiple indices are combined with |.
#### Deduplication → keeps one row per (Stock Symbol, Stock Market).
#### Simple, maintainable structure — easy to extend for ISIN, sector, country later.

In [1]:
import pandas as pd
from pathlib import Path
from time import sleep
from io import StringIO
import requests

import requests
from pathlib import Path
from time import sleep
from io import StringIO
import pandas as pd

# =========================
# Directories
# =========================
RAW_DIR = Path("../Data/raw_csv")
RAW_DIR.mkdir(parents=True, exist_ok=True)

# =========================
# Index URLs
# =========================
URLS = {
    # US (DataHub)
    "S&P500": "https://datahub.io/core/s-and-p-500-companies/r/constituents.csv",
    "NASDAQ": "https://datahub.io/core/nasdaq-listings/r/nasdaq-listed-symbols.csv",
    "NYSE": "https://datahub.io/core/nyse-other-listings/_r/-/data/nyse-listed.csv",
    # INDIA (NIFTY)
    "NIFTY50": "https://www.niftyindices.com/IndexConstituent/ind_nifty50list.csv",
    "NIFTY500": "https://www.niftyindices.com/IndexConstituent/ind_nifty500list.csv",
}

# Browser-like headers (CRITICAL for NIFTY)
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    ),
    "Accept": "text/csv,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Referer": "https://www.niftyindices.com/",
    "Connection": "keep-alive",
}

# =========================
# Download with retries
# =========================
def download_with_retry(session, name, url, retries=3):
    for attempt in range(1, retries + 1):
        try:
            print(f"Downloading {name} (attempt {attempt})")
            resp = session.get(url, headers=HEADERS, timeout=120)
            resp.raise_for_status()
            return resp.content
        except Exception as e:
            print(f"⚠️ Attempt {attempt} failed for {name}: {e}")
            sleep(5 * attempt)
    raise RuntimeError(f"❌ Failed after {retries} attempts: {name}")

# =========================
# Download all raw CSVs
# =========================
def download_raw():
    with requests.Session() as session:
        for name, url in URLS.items():
            content = download_with_retry(session, name, url)
            file_path = RAW_DIR / f"{name}.csv"
            file_path.write_bytes(content)
            print(f"✅ Saved raw file → {file_path}")

        # -------------------------
        # Dow Jones (Wikipedia)
        # -------------------------
        print("Downloading DOW_JONES from Wikipedia API")
        WIKI_API = "https://en.wikipedia.org/w/api.php"
        params = {"action": "parse", "page": "Dow_Jones_Industrial_Average",
                  "prop": "text", "format": "json"}
        r = session.get(WIKI_API, params=params, headers={"User-Agent": "IndexDataCollector/1.0"}, timeout=60)
        r.raise_for_status()
        html = r.json()["parse"]["text"]["*"]
        tables = pd.read_html(StringIO(html))
        djia_table = next(t for t in tables if {"Company", "Symbol"}.issubset(set(t.columns)))
        djia_df = djia_table[["Company", "Symbol"]]
        djia_df.columns = ["Stock Name", "Stock Symbol"]
        djia_path = RAW_DIR / "DOW_JONES.csv"
        djia_df.to_csv(djia_path, index=False)
        print(f"✅ Saved raw file → {djia_path}")

 

# =========================
# Download Dow Jones via Wikipedia
# =========================
def download_dow_jones():
    print("⬇️ Downloading DOW_JONES from Wikipedia API")
    WIKI_API = "https://en.wikipedia.org/w/api.php"
    headers = {"User-Agent": "IndexDataCollector/1.0"}
    params = {"action": "parse", "page": "Dow_Jones_Industrial_Average",
              "prop": "text", "format": "json"}
    r = requests.get(WIKI_API, params=params, headers=headers, timeout=60)
    r.raise_for_status()
    html = r.json()["parse"]["text"]["*"]
    tables = pd.read_html(StringIO(html))
    djia_table = next(t for t in tables if {"Company", "Symbol"}.issubset(set(t.columns)))
    djia_df = djia_table[["Company", "Symbol"]]
    djia_df.columns = ["Stock Name", "Stock Symbol"]
    path = RAW_DIR / "DOW_JONES.csv"
    djia_df.to_csv(path, index=False)
    print(f"✅ Saved DOW_JONES → {path}")
    return djia_df
    
RAW_DIR = Path("../Data/raw_csv")
OUT_DIR = Path("../Data/normalized_csv")
OUT_DIR.mkdir(exist_ok=True)

# =========================
# Process Index CSVs
# =========================
def process_sp500():
    df = pd.read_csv(RAW_DIR / "S&P500.csv")
    return pd.DataFrame({
        "Stock Name": df["Security"],
        "Stock Symbol": df["Symbol"],
        "Stock Market": "NYSE/NASDAQ",
        "INDEX": "S&P 500"
    })

def process_nasdaq():
    df = pd.read_csv(RAW_DIR / "NASDAQ.csv")
    return pd.DataFrame({
        "Stock Name": df["Company Name"],
        "Stock Symbol": df["Symbol"],
        "Stock Market": "NASDAQ",
        "INDEX": "NASDAQ"
    })

def process_nyse():
    csv_path = RAW_DIR / "NYSE.csv"
    if not csv_path.exists():
        print("⚠️ NYSE.csv not found, skipping NYSE index")
        return pd.DataFrame(columns=["Stock Name","Stock Symbol","Stock Market","INDEX"])
    df = pd.read_csv(csv_path)
    return pd.DataFrame({
        "Stock Name": df["Company Name"],
        "Stock Symbol": df["ACT Symbol"],
        "Stock Market": "NYSE",
        "INDEX": "NYSE"
    })

def process_nifty(index_name):
    csv_path = RAW_DIR / f"{index_name}.csv"
    if not csv_path.exists():
        print(f"⚠️ {index_name}.csv not found, skipping")
        return pd.DataFrame(columns=["Stock Name","Stock Symbol","Stock Market","INDEX"])
    df = pd.read_csv(csv_path)
    df = df[["Company Name", "Symbol"]]
    return pd.DataFrame({
        "Stock Name": df["Company Name"],
        "Stock Symbol": df["Symbol"],
        "Stock Market": "NSE",
        "INDEX": index_name.replace("NIFTY", "NIFTY ")
    })

def process_dow_jones():
    df = pd.read_csv(RAW_DIR / "DOW_JONES.csv")
    return pd.DataFrame({
        "Stock Name": df["Stock Name"],
        "Stock Symbol": df["Stock Symbol"],
        "Stock Market": "NYSE/NASDAQ",
        "INDEX": "DOW JONES"
    })

# =========================
# Main
# =========================
def main():
    download_raw()
    download_dow_jones()

    # Process all indices
    datasets = [
        process_sp500(),
        process_nasdaq(),
        process_nyse(),
        process_nifty("NIFTY50"),
        process_nifty("NIFTY500"),
        process_dow_jones(),
    ]

    # Combine, deduplicate, and aggregate
    final_df = pd.concat(datasets, ignore_index=True)
    final_df.dropna(inplace=True)
    final_df = (final_df.groupby(["Stock Symbol", "Stock Market"], as_index=False)
                .agg({"Stock Name": "first",
                      "INDEX": lambda x: "|".join(sorted(set(x)))}))

    out_file = OUT_DIR / "ALL_INDEX_STOCKS.csv"
    final_df.to_csv(out_file, index=False)

    print(f"✅ Normalized file created → {out_file}")
    print(f"Total rows: {len(final_df)}")

if __name__ == "__main__":
    main()


✅ Saved raw file → ..\Data\raw_csv\S&P500.csv
✅ Saved raw file → ..\Data\raw_csv\NASDAQ.csv
✅ Saved raw file → ..\Data\raw_csv\NYSE.csv
✅ Saved raw file → ..\Data\raw_csv\NIFTY50.csv
✅ Saved raw file → ..\Data\raw_csv\NIFTY500.csv
✅ Saved raw file → ..\Data\raw_csv\DOW_JONES.csv
⬇️ Downloading DOW_JONES from Wikipedia API
✅ Saved DOW_JONES → ..\Data\raw_csv\DOW_JONES.csv
✅ Normalized file created → ..\Data\normalized_csv\ALL_INDEX_STOCKS.csv
Total rows: 9104
